In [135]:
import requests
import json
import os
import pandas as pd
import shutil
from bs4 import BeautifulSoup

In [70]:
# scraping
ids_to_scrape = [113009062564109638,113009132396502015,113009893061956633,113013201864537613,113015097117461479]

In [110]:
def fetch_posts(max_id):
    cookies = {
        '__cflb': '0H28vTPqhjwKvpvovPffsJ3cyNVudUuwhqw854QkRoV',
        '__cf_bm': 'yc3dJp_Owcxx8TliDBQO9jOj63HF5qUcxTkzmCppWcw-1724790574-1.0.1.1-MagjG2kesFhENJcJWtQ1JLBUJ.2nrUFkwnE3l6vzsSXqtc_zzozNKeXmVJad1fTtC14.Hji4Pzw1MkuYAg3fjA',
        '_cfuvid': '4U4YzdUV7VAIxlbNL16Ly0ttJrEjXdjZghgZC9.PGH0-1724789320601-0.0.1.1-604800000',
        '_tq_id.TV-5427368145-1.4081': 'f62a0df10e808ce7.1724789321.0.1724790588..',
        'mp_15c0cd079bcfa80cd935f3a1b8606b48_mixpanel': '%7B%22distinct_id%22%3A%20%2219195746ceaa4f-047fa0e479f65c-42272e3d-1fa400-19195746ceb138a%22%2C%22%24device_id%22%3A%20%2219195746ceaa4f-047fa0e479f65c-42272e3d-1fa400-19195746ceb138a%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%7D',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:129.0) Gecko/20100101 Firefox/129.0',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.5',
        # 'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Referer': 'https://truthsocial.com/@realDonaldTrump',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Connection': 'keep-alive'
    }

    params = {
        'exclude_replies': 'true',
        'with_muted': 'true',
        'max_id':max_id
    }

    response = requests.get(
        'https://truthsocial.com/api/v1/accounts/107780257626128497/statuses',
        params=params,
        cookies=cookies,
        headers=headers,
    )

    return response.json()

def download_image(url,out_path):
    response = requests.get(url, stream=True)
    with open(out_path, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

In [72]:
for id in ids_to_scrape:
    data = fetch_posts(id)
    with open('truth-social_receipts/' + str(id) + '.json','w') as out_file:
        json.dump(data,out_file)

In [128]:
# analyzing
receipts = ['truth-social_receipts/' + file for file in os.listdir('truth-social_receipts')]

In [129]:
data_chunks = []
for receipt in receipts:
    with open(receipt,'r') as in_file:
        data_chunks.append(json.load(in_file))

In [130]:
df = pd.concat([pd.DataFrame(data) for data in data_chunks])

In [131]:

df = df[['id','created_at','in_reply_to_id','quote_id','url','content','media_attachments','replies_count','reblogs_count','favourites_count']]

In [132]:
df.reset_index(inplace=True,drop=True)

In [133]:
# for media_entry in df['media_attachments']:
#     if len(media_entry) > 0:
#         media = media_entry[0]
#         if media['type'] == 'image':
#             download_image(media['url'],'media/' + media['id'] + '.jpg')
#         else:
#             download_image(media['url'],'media/' + media['id'] + '.mp4')

In [159]:
df['text_content'] = df['content'].apply(lambda x: BeautifulSoup(x).get_text())

In [161]:
df.to_csv('trump-posts.csv',index=False)